In [4]:
from vit import vit_base

In [5]:
model = vit_base(patch_size=8)

In [6]:
import torch
model.load_state_dict(torch.load('/viscam/u/redfairy/pretrained_weights/dino_vitbase8_pretrain.pth'))

<All keys matched successfully>